In [5]:
import pandas
import os
from datetime import datetime

current_path = os.path.abspath('')
df1 = pandas.read_excel(current_path + '/../data/集思录.xlsx')
df2 = pandas.read_excel(current_path + '/../data/上市发行.xlsx')
df3 = pandas.read_excel(current_path + '/../data/发行结果.xlsx')
df4 = pandas.read_excel(current_path + '/../data/上市表现.xlsx')
df5 = pandas.read_excel(current_path + '/../data/正股价格.xlsx', converters={'股票代码':str})

df1['可转债代码'] = df1['证券代码'].fillna('000000.SH').map(lambda x:[c.strip() for c in x[1:-1].replace('\'','').split(',')][1])
df2['可转债代码'] = df2['转债代码'].fillna('000000.SH').map(lambda x:[c.strip() for c in x.split('.')][0])
df3['可转债代码'] = df3['代码'].fillna('000000.SH').map(lambda x:[c.strip() for c in x.split('.')][0])
df4['可转债代码'] = df4['代码'].fillna('000000.SH').map(lambda x:[c.strip() for c in x.split('.')][0])

df = pandas.merge(df1,df2, on='可转债代码', how='left')
df = pandas.merge(df,df3, on='可转债代码', how='left')
df = pandas.merge(df,df4, on='可转债代码', how='left')

for idx, row in df.iterrows():
    # 第一个是正股代码， 第二个是可转债代码
    symbol = [c.strip() for c in row['证券代码'][1:-1].replace('\'','').split(',')]
    name = [c.strip() for c in row['证券名称'][1:-1].replace('\'','').split(',')]
    # 可转债上市日期
    listing_date = row['上市公告日期']
    # 可转债配售股权登记日 (T-1日)
    registration = row['股权登记日']
    #print(symbol, name, listing_date, registration)
    # 网上发行日 (T日)
    day_t = row['网上发行日期_x'].strftime("%Y-%m-%d")
    # 发行公告日 (T-2日)
    day_t_2 = row['发行公告日'].strftime("%Y-%m-%d")

    print(symbol, name, day_t_2, registration, day_t, listing_date)
    price = df5[(df5['股票代码']==symbol[0]) & (df5['日期']==day_t)]
    #print(price)
    for idx in price.index:
        df.loc[df['可转债代码']==symbol[1], 'T日开盘'] = price.at[idx,'开盘']
        df.loc[df['可转债代码']==symbol[1], 'T日收盘'] = price.at[idx,'收盘']
        break
    price = df5[(df5['股票代码']==symbol[0]) & (df5['日期']==registration)]
    for idx in price.index:
        df.loc[df['可转债代码']==symbol[1], 'T-1日开盘'] = price.at[idx,'开盘']
        df.loc[df['可转债代码']==symbol[1], 'T-1日收盘'] = price.at[idx,'收盘']
        break
    price = df5[(df5['股票代码']==symbol[0]) & (df5['日期']==day_t_2)]
    for idx in price.index:
        df.loc[df['可转债代码']==symbol[1], 'T-2日开盘'] = price.at[idx,'开盘']
        df.loc[df['可转债代码']==symbol[1], 'T-2日收盘'] = price.at[idx,'收盘']
        break
    
    
    
df.to_excel(current_path + '/../data/data.xlsx', index=False)


['001203', '127070'] ['大中矿业', '大中转债'] 2022-08-13 2022-08-16 2022-08-17 2022-10-11
['001203', '127070'] ['大中矿业', '大中转债'] 2022-08-14 2022-08-16 2022-08-17 2022-10-11
['002997', '127065'] ['瑞鹄模具', '瑞鹄转债'] 2022-06-18 2022-06-21 2022-06-22 2022-07-27
['002997', '127065'] ['瑞鹄模具', '瑞鹄转债'] 2022-06-19 2022-06-21 2022-06-22 2022-07-27
['601686', '113058'] ['友发集团', '友发转债'] 2022-03-26 2022-03-29 2022-03-30 2022-04-26
['601686', '113058'] ['友发集团', '友发转债'] 2022-03-27 2022-03-29 2022-03-30 2022-04-26
['603605', '113634'] ['珀莱雅', '珀莱转债'] 2021-12-04 2021-12-07 2021-12-08 2022-01-04
['603605', '113634'] ['珀莱雅', '珀莱转债'] 2021-12-05 2021-12-07 2021-12-08 2022-01-04
['002568', '127046'] ['百润股份', '百润转债'] 2021-09-25 2021-09-28 2021-09-29 2021-11-03
['002568', '127046'] ['百润股份', '百润转债'] 2021-09-26 2021-09-28 2021-09-29 2021-11-03
['002091', '127040'] ['江苏国泰', '国泰转债'] 2021-07-03 2021-07-06 2021-07-07 2021-08-10
['002091', '127040'] ['江苏国泰', '国泰转债'] 2021-07-04 2021-07-06 2021-07-07 2021-08-10
['603225', '113623